<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/NLP/4-8-LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences


IMDB 영화 리뷰 데이터셋을 로드하고, 전처리 과정을 수행합니다.

In [2]:
# Hyperparameters
max_features = 10000  # 사용할 단어의 최대 개수
max_len = 200  # 패딩 후 시퀀스의 길이
embedding_dim = 128

# IMDB 데이터셋 로드
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# 시퀀스 길이를 일정하게 맞추기 위해 패딩 추가
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


LSTM 레이어와 드롭아웃을 포함한 모델을 정의합니다.

In [3]:
def create_model():
    model = Sequential()

    # 임베딩 층: 단어 인덱스를 밀집 벡터로 매핑
    model.add(Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=max_len))

    # LSTM 층: 순차 데이터 처리
    model.add(LSTM(128, return_sequences=False))

    # 드롭아웃 층: 과적합 방지
    model.add(Dropout(0.5))

    # 출력 층: 이진 분류 (긍정/부정)
    model.add(Dense(1, activation='sigmoid'))

    return model

# 모델 생성
model = create_model()

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

학습 데이터를 사용하여 모델을 훈련합니다. 검증 데이터로 테스트 데이터를 사용하여 학습 중 모델의 성능을 확인합니다.

In [4]:
# 모델 학습
history = model.fit(
    x_train,
    y_train,
    epochs=5,
    batch_size=64,
    validation_data=(x_test, y_test),
    verbose=2
)


Epoch 1/5
391/391 - 15s - 39ms/step - accuracy: 0.7688 - loss: 0.4838 - val_accuracy: 0.7554 - val_loss: 0.4984
Epoch 2/5
391/391 - 8s - 19ms/step - accuracy: 0.8810 - loss: 0.2967 - val_accuracy: 0.8690 - val_loss: 0.3105
Epoch 3/5
391/391 - 11s - 29ms/step - accuracy: 0.9154 - loss: 0.2230 - val_accuracy: 0.8746 - val_loss: 0.3309
Epoch 4/5
391/391 - 8s - 21ms/step - accuracy: 0.9404 - loss: 0.1622 - val_accuracy: 0.8655 - val_loss: 0.3752
Epoch 5/5
391/391 - 10s - 27ms/step - accuracy: 0.9524 - loss: 0.1311 - val_accuracy: 0.8586 - val_loss: 0.3984


테스트 데이터를 사용하여 학습된 모델을 평가합니다.

In [5]:
# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test Accuracy: {test_acc:.4f}')


782/782 - 4s - 5ms/step - accuracy: 0.8586 - loss: 0.3984
Test Accuracy: 0.8586


새로운 리뷰에 대해 감성을 예측해보는 단계입니다. 아래는 샘플 입력 데이터에 대해 모델이 예측하는 코드입니다.

In [7]:
# 예측 예제
sample_text = ["This movie was absolutely fantastic! The performances were breathtaking and the story was captivating."]

# 텍스트를 토큰화하고 인덱스로 변환 (예시)
# 실제로는 적절한 토크나이저를 사용해야 함 (e.g., spaCy, nltk)
sample_tokens = sample_text[0].split()
word_index = {word: index for index, word in enumerate(sample_tokens)}
sample_indices = [word_index[word] for word in sample_tokens]

# 예제 데이터를 위한 패딩 처리
sample_input = pad_sequences([sample_indices], maxlen=max_len)

# 예측
prediction = model.predict(sample_input)
print(f'Predicted sentiment: {"Positive" if prediction > 0.5 else "Negative"}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Predicted sentiment: Positive
